# Face Recognition Model Training
## Overview
This notebook trains a face recognition model using:
- OpenCV for face detection (Haar Cascade).
- TensorFlow (VGGFace) for feature extraction.
- Cosine Similarity for recognition.
The model is trained on a custom dataset and saved as `face_database.pkl`.

## Setup
Install dependencies and import libraries.

In [3]:

!pip install tensorflow==2.9.0
!pip install keras==2.9
!pip install face_recognition
!pip install keras_applications==1.0.8
!pip install keras-vggface
!pip install h5py
!pip install matplotlib
!pip install scikit-learn
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 97.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 48.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.25
    Uninstalling flatbuffers-24.3.25:
      Successfully uninstalled flatbuffers-24.3.25
  Attempting unins

In [4]:
import os
import cv2
import numpy as np
import pickle
import face_recognition
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omchoksi04/celebrity-faceid-dataset")

print("Path to dataset files:", path)


In [ ]:

# Define dataset path
dataset_path = path
model_path = "face_embeddings.pkl"


In [16]:
def extract_faces(image):
    """ Detect faces in an image using face_recognition """
    faces = face_recognition.face_locations(image)
    if faces:
        return faces
    return None

In [17]:
def get_embeddings():
    """ Extract face embeddings from dataset """
    encodings = []
    names = []
    
    for person in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person)
        if os.path.isdir(person_path):
            for img_name in os.listdir(person_path):
                img_path = os.path.join(person_path, img_name)
                img = cv2.imread(img_path)
                rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                faces = extract_faces(rgb_img)
                if faces:
                    encoding = face_recognition.face_encodings(rgb_img, faces)
                    if encoding:
                        encodings.append(encoding[0])
                        names.append(person)
    
    return encodings, names



In [18]:
# Train & Save Model
def train():
    print("Training on dataset...")
    encodings, names = get_embeddings()
    data = {"encodings": encodings, "names": names}
    with open(model_path, "wb") as f:
        pickle.dump(data, f)
    print("Model saved!")

train()



Training on dataset...
Model saved!


In [19]:
# Load Model
def load_trained_model():
    with open(model_path, "rb") as f:
        data = pickle.load(f)
    return data["encodings"], data["names"]

def recognize_face(frame):
    encodings, names = load_trained_model()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, faces)

    for face_encoding, face_location in zip(face_encodings, faces):
        distances = cosine_similarity([face_encoding], encodings)
        best_match = np.argmax(distances)
        confidence = distances[0][best_match]
        name = names[best_match] if confidence > 0.6 else "Unknown"
        
        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    return frame


2025-03-13 11:19:38.692 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:19:38.867 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-13 11:19:38.868 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:19:38.869 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:19:38.869 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:19:38.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:19:38.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:19:38.872 Session state does not 